# Programming for Data Science and Artificial Intelligence

## Deep Learning - NLP + TorchText + Embedding

Here we shall improve the previous one by adding:

Improve the learning
- pre-trained word embeddings (**) (improve accuracy by around 20)
- changed optimizer to Adam (make the thing learn faster)

Improve efficiency
- packed padded sequences in RNN to save computations and also ask the RNN to ignore padding (++) (this is the deal breaker; without this, my accuracy is 50) (https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch)
- put padding_idx in embedding layer to save computations (no hit to accuracy but good practice to do)

In [1]:
import torchtext
import torch
from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cuda


### Loading the dataset

In [2]:
#uncomment this if you are not using puffer
import os
os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

from torchtext.datasets import IMDB
train_iter, test_iter = IMDB(split=('train', 'test'))

### Tokenizing

In [3]:
#pip install spacy
#python -m spacy download en_core_web_sm
from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
tokens = tokenizer("We are learning torchtext in U.K.!")  #some test
tokens

['We', 'are', 'learning', 'torchtext', 'in', 'U.K.', '!']

### Text to integers

In [4]:
from torchtext.vocab import build_vocab_from_iterator
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=['<unk>', '<pad>', '<bos>', '<eos>'])
vocab.set_default_index(vocab["<unk>"])

In [5]:
#see some example
vocab(['here', 'is', 'a', 'unknownword', 'a'])

[164, 11, 8, 0, 8]

In [6]:
len(vocab)

121068

### ** FastText Embeddings **

We will first download the pre-trained vectors, here I am using FastText.  Then we will get all the FastText embeddings that exist in the vocab.  

**Small Intro to Embeddings**

1. *Word2Vec* - the first efficient word embedding, trained on Continuous Bag-of-words (CBOW) and SkipGram (SG).  The limitations include: (1) works only with local window information, not the whole document, (2) no subword information (prefix, suffix, etc.), (3) cannot handle OOV words, and (4) do not handle context.

First three problems were addressed by GloVe and FastText, and last one has been resolved by Elmo and BERT.

<img src = "../figures/word2vec.png" width=300>

2. *GloVe* - particularly adresses problem no. 1 which uses co-occurrence statistics of the whole corpus.  For example, given words i=ice, j=steam, we want to study a ratio of occurrence probabilities with some probe word k=solid as this figure:

<img src = "../figures/glove.png" width=300>

3. *FastText* - addresses problem no. 2 and 3.  Uses the skipgram arhitecture to train but with the following improvements:  (1) faster and simpler to train, (2) consider subwords as ngrams (If we consider the word “what” and use n=3 or tri-grams, the word would be represented by the character n-grams: <”wh”,”wha”,”hat”,”at”>. < and > are special symbols that are added at the start and end of each word.), (3) it can generate embeddings from OOV thanks to the ngrams.  An OOV word vector can be built with the average vector representation of its n-grams.     Big disadvantage is its high memory requirements.

4. *ElMo* - given same word "stick" can have different meanings.  By using a bi-directional LSTM, ElMo was able to understand not only the next words, but also the preceding ones.  Also can work like FastText on subwords and do not suffer OOV problem.

In [7]:
#https://github.com/pytorch/text/issues/1350
from torchtext.vocab import FastText
fast_vectors = FastText()

In [8]:
fast_embedding = fast_vectors.get_vecs_by_tokens(vocab.get_itos()).to(device)
# vocab.get_itos() returns a list of strings (tokens), where the token at the i'th position is what you get from doing vocab[token]
# get_vecs_by_tokens gets the pre-trained vector for each string when given a list of strings
# therefore pretrained_embedding is a fully "aligned" embedding matrix

In [9]:
fast_embedding.shape   #we have 121068 vocabs, each with a 300d fasttext embedding

torch.Size([121068, 300])

### Defining hyperparameters

In [10]:
#model hyperparameters
input_dim = len(vocab)
hidden_dim = 256
embed_dim = 300 #**<----cannot change if you are using FastText because that's the dimension of FastText
output_dim = 1
pad_idx = vocab['<pad>'] #++<----making sure our embedding layer ignores pad

#training hyperparameters
batch_size = 64
num_epochs = 5
lr=0.0001

### Batch Iterator

In [11]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: 1 if x == 'pos' else 0

In [12]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence #++

def collate_batch(batch):
    label_list, text_list, length_list = [], [], []
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        length_list.append(processed_text.size(0))  #++<-----packed padded sequences require length
    #criterion expects float labels
    return torch.tensor(label_list, dtype=torch.float64), pad_sequence(text_list), torch.tensor(length_list, dtype=torch.int64)

In [13]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

train_iter, test_iter = IMDB()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_loader = DataLoader(split_train_, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_batch)
valid_loader = DataLoader(split_valid_, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=batch_size,
                             shuffle=True, collate_fn=collate_batch)

### ++ About pack padded sequence ++
By packing the padded sequence, the RNN (RNN, LSTM, GRU) does not need to do unnecessary computations.

In [14]:
a = [torch.tensor([1,2,3]), torch.tensor([3,4])]
b = torch.nn.utils.rnn.pad_sequence(a)
b

tensor([[1, 3],
        [2, 4],
        [3, 0]])

In [15]:
c = torch.nn.utils.rnn.pack_padded_sequence(b, lengths=[3,2])
c

PackedSequence(data=tensor([1, 3, 2, 4, 3]), batch_sizes=tensor([2, 2, 1]), sorted_indices=None, unsorted_indices=None)

### Build the model

In [16]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        #put padding_idx so asking the embedding layer to ignore padding
        self.embedding = nn.Embedding(input_dim, embed_dim, padding_idx=pad_idx)
        self.rnn = nn.RNN(embed_dim, hidden_dim, num_layers=1)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        #text = [sent len, batch size]
        embedded = self.embedding(text)
        
        #++ pack sequence ++
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'), enforce_sorted=False)
        
        #embedded = [sent len, batch size, embed dim]
        packed_output, hn = self.rnn(packed_embedded)  #if no h0, all zeroes
        
        #++ unpack in case we need to use it ++
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        
        #output = [sent len, batch size, hidden dim]
        #hidden = [1, batch size, hidden dim]
        
        #assert torch.equal(output[-1,:,:], hn.squeeze(0))
        return self.fc(hn.squeeze(0))

### Training

In [17]:
import torch.optim as optim

model = RNN().to(device)
model.embedding.weight.data = fast_embedding #**<------applied the fast text embedding as the initial weights
optimizer = optim.Adam(model.parameters(), lr=lr) #<----changed to Adam
criterion = nn.BCEWithLogitsLoss() #combine sigmoid with binary cross entropy

In [18]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [19]:
def train(model, loader, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train() #useful for batchnorm and dropout
    for i, (label, text, text_length) in enumerate(loader): 
        label = label.to(device) #(batch_size, )
        text = text.to(device) #(sent len, batch_size)
                
        #predict
        predictions = model(text, text_length).squeeze(1) #output by the fc is (batch_size, 1), thus need to remove this 1
        
        #calculate loss
        loss = criterion(predictions, label)
        acc = binary_accuracy(predictions, label)
        
        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
                
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [20]:
def evaluate(model, loader, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for i, (label, text, text_length) in enumerate(loader): 
            label = label.to(device) #(batch_size, )
            text = text.to(device) #(sent len, batch_size)

            predictions = model(text, text_length).squeeze(1) 
            
            loss = criterion(predictions, label)
            acc = binary_accuracy(predictions, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(loader), epoch_acc / len(loader)

### Putting everything together

In [21]:
best_valid_loss = float('inf')

for epoch in range(num_epochs):

    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'models/fasttext.pt')
    
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Train Loss: 0.666 | Train Acc: 63.53%
	 Val. Loss: 0.691 |  Val. Acc: 53.28%
Epoch: 02 | Train Loss: 0.678 | Train Acc: 59.72%
	 Val. Loss: 0.674 |  Val. Acc: 61.79%
Epoch: 03 | Train Loss: 0.638 | Train Acc: 66.38%
	 Val. Loss: 0.516 |  Val. Acc: 77.37%
Epoch: 04 | Train Loss: 0.500 | Train Acc: 76.39%
	 Val. Loss: 0.667 |  Val. Acc: 57.32%
Epoch: 05 | Train Loss: 0.507 | Train Acc: 75.52%
	 Val. Loss: 0.435 |  Val. Acc: 80.97%


In [22]:
model.load_state_dict(torch.load('models/fasttext.pt'))
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.419 | Test Acc: 81.81%


### Test on some random reviews

In [30]:
test_str = "This is Chaky.  This movie is really good good good"
text = torch.tensor(text_pipeline(test_str)).to(device)
text

tensor([68, 11,  0,  6,  0, 68, 24, 11, 79, 62, 62, 62], device='cuda:0')

In [31]:
text = text.reshape(-1, 1)  #because batch_size is 1

In [32]:
text_length = torch.tensor([text.size(0)]).to(dtype=torch.int64)

In [33]:
text.shape

torch.Size([12, 1])

In [34]:
text_length.shape

torch.Size([1])

In [35]:
def predict(text, text_pipeline):
    model.eval()
    with torch.no_grad():
        output = model(text, text_length).squeeze(1)
        rounded_preds = torch.round(torch.sigmoid(output))
        return rounded_preds

In [36]:
predict(text, text_pipeline)  #quite accurate!!! try change to opposite

tensor([1.], device='cuda:0')

### Practice

- Try to turn off FastText embedding and see the accuracy.  For me, the accuracy reduced by around 10 to 20%
- Try to change Adam back to SGD.
- Try not to pack sequence and see what happens
- Try change your personal review and see whether your model can do well

### Trivials

If you don't like to pad, you can either use batch_size=1, or group samples by length.

Next class, let's try LSTM which is a better variant of RNN and see whether the accuracy improves.